<div align="center">

# Play Ground Series S3 E26

## 💉 Predicting outcomes of patients with cirrhosis 💉

### Welcome to my kaggle notebook! For this Episode of the Series, is to use a multi-class approach to predict the the outcomes of patients with cirrhosis.

</div>

## Goal: For this Episode of the Series, is to use a multi-class approach to predict the the outcomes of patients with cirrhosis
i think this is my most indepth notebook yet, i have tried to explain everything i did in detail, i hope you enjoy it and learn something from it.

In [ ]:
#TODO:
#feature engineering then outliar detection

In [ ]:
# Standard library imports
import pandas as pd
import numpy as np


# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# ML imports

In [94]:
df_train = pd.read_csv(r"..\data\train.csv")
df_test = pd.read_csv(r"..\data\test.csv")
df_original = pd.read_csv(r"..\data\cirrhosis.csv")

# Rename ID column to id for consistency and concat with train data
df_original = df_original.rename(columns={"ID":"id"})

#combine train and original data
combined_train_df = pd.concat([df_train, df_original],ignore_index=True)
combined_train_df = combined_train_df.sample(frac=1, random_state=1).reset_index(drop=True)



In [ ]:
# checking for null values after combining train and original data

display(df_test.head())
display(df_original.head())
display(combined_train_df.head(10))
display(combined_train_df.isna().sum())
display(combined_train_df.value_counts('Stage'))



In [96]:
# impute missing numerical values with knn imputer

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
numerical_columns = combined_train_df.select_dtypes(include=[np.number]).columns
combined_train_df_imputed = combined_train_df.copy()
combined_train_df_imputed[numerical_columns] = imputer.fit_transform(combined_train_df[numerical_columns])
display(combined_train_df_imputed.isna().sum())
display(combined_train_df_imputed.value_counts('Stage'))


id                 0
N_Days             0
Drug             106
Age                0
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol        0
Albumin            0
Copper             0
Alk_Phos           0
SGOT               0
Tryglicerides      0
Platelets          0
Prothrombin        0
Stage              0
Status             0
dtype: int64

Stage
3.0    3311
4.0    2848
2.0    1744
1.0     418
2.4       1
3.6       1
Name: count, dtype: int64

In [97]:
combined_train_df_imputed['Stage'] = combined_train_df_imputed['Stage'].round().astype(int)
combined_train_df_imputed.value_counts('Stage')

Stage
3    3311
4    2849
2    1745
1     418
Name: count, dtype: int64

In [98]:
# impute missing values with most frequent value

for col in combined_train_df_imputed.select_dtypes(include=['object']).columns:
    most_frequent = combined_train_df_imputed[col].mode()[0]
    combined_train_df_imputed[col].fillna(most_frequent, inplace=True)
combined_train_df_imputed.isna().sum()

id               0
N_Days           0
Drug             0
Age              0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
Stage            0
Status           0
dtype: int64

In [ ]:
def optimize_df(df):
    """
    Optimizes a pandas DataFrame's memory usage by downcasting 
    numerical columns to the smallest possible types and converting 
    object columns to categorical if appropriate.

    Parameters:
    df (pandas.DataFrame): The DataFrame to optimize.

    Returns:
    pandas.DataFrame: The optimized DataFrame.
    """

    # Downcast numerical columns to save memory
    int_columns = df.select_dtypes(include=['int64']).columns
    float_columns = df.select_dtypes(include=['float64']).columns

    # Downcast int64 columns to the smallest integer dtype
    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')

    # Downcast float64 columns to the smallest float dtype
    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast='float')

    # # Convert object columns to categorical if they have a relatively low cardinality
    # object_columns = df.select_dtypes(include=['object']).columns
    # for col in object_columns:
    #     num_unique_values = len(df[col].unique())
    #     num_total_values = len(df[col])
    #     if num_unique_values / num_total_values < 0.5:  # Threshold to decide if the column should be categorical
    #         df[col] = df[col].astype('category')

    return df

# Example usage:
combined_train_df_imputed = optimize_df(combined_train_df_imputed)

In [ ]:
def analyze_dataframe(df):
    """
    Analyze a pandas DataFrame and provide a summary of its characteristics.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    print("DataFrame Information:")
    print("______________________")
    display(df.info(verbose=True, show_counts=True))
    print("\n")
    
    print("DataFrame Head:")
    print("______________________")
    display(df.head())
    print("\n")

    print("DataFrame Tail:")
    print("______________________")
    display(df.tail())
    print("\n")

    print("DataFrame Description:")
    print("______________________")
    display(df.describe().T)
    print("\n")

    print("Number of Null Values:")
    print("______________________")
    display(df.isnull().sum())
    print("\n")

    print("Number of Duplicated Rows:")
    print("______________________")
    display(df.duplicated().sum())
    print("\n")

    print("Number of Unique Values:")
    print("______________________")
    display(df.nunique())
    print("\n")

    print("DataFrame Shape:")
    print("______________________")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
    

analyze_dataframe(combined_train_df_imputed)

In [ ]:
combined_train_df_imputed.value_counts('Stage')

## Data points interpretation that needed further explanation
___

1. **Ascites**: Ascites is a medical term for an abnormal fluid buildup in the abdominal cavity, specifically in the peritoneal space. This syndrome is frequently caused by underlying health conditions such as liver disease, cirrhosis, heart failure, cancer, or infections.

2. **Hepatomegaly**: Hepatomegaly refers to an enlargement of the liver. It is a non-specific medical sign, having many causes, which can broadly be broken down into infection, hepatic tumors, and metabolic disorder.

3. **Spiders**: In medical terms, 'spiders' could refer to spider angiomas (also known as spider nevi), which are common skin lesions consisting of central arterioles surrounded by many smaller vessels due to high estrogen levels and may occur in any condition with high estrogen states such as cirrhosis.

4. **Edema**: Edema is swelling caused by fluid trapped in your body’s tissues, most often in your feet, ankles, and legs. Your healthcare provider will test your edema by pressing their finger into the swollen area (pitting) to identify how much fluid is in your tissues (grade).

5. **Bilirubin**: Bilirubin is a reddish-yellow water-insoluble pigment that is formed by the breakdown of heme, is excreted in a water-soluble form by liver cells into bile, and occurs in blood and urine especially in diseased states.

6. **Cholesterol**: Cholesterol is a waxy, fat-like substance that's found in all the cells in your body. Your body needs some cholesterol to make hormones, vitamin D, and substances that help you digest foods.

7. **Albumin**: Albumin is a type of protein that is found in your blood. It's produced by your liver and serves several important functions in the body. One of its main roles is to help maintain the right amount of water in your blood and tissues.

8. **Copper**: Copper, an essential mineral, is naturally present in some foods and is available as a dietary supplement. It is a cofactor for several enzymes (known as cuproenzymes) involved in energy production, iron metabolism, 
neuropeptide activation, connective tissue synthesis, and neurotransmitter synthesis.

9. **Alk_Phos**: Alkaline phosphatase (ALP) is an enzyme that’s found throughout your body. An enzyme is a type of protein in a cell that acts as a catalyst and allows certain bodily processes to happen.

10. **SGOT**: Serum glutamic oxaloacetic transaminase (SGOT), also known as aspartate aminotransferase (AST), is an enzyme that is normally present in liver and heart cells. SGOT is released into blood when the liver or heart is damaged.

11. **Triglycerides**: Triglycerides are a type of fat (lipid) found in your blood. When you eat, your body converts any calories it doesn't need to use right away into triglycerides. The triglycerides are stored in your fat cells. Later, hormones release triglycerides for energy between meals.

## Normal values of the data points
____
1. **Bilirubin**:
   - Total: 0.1 to 1.2 mg/dL

2. **Cholesterol**:
   - Total: Less than 200 mg/dL

3. **Albumin**:
   - 3.4 to 5.4 g/dL

4. **Copper**:
   - 70 to 150 mcg/dL

5. **Alkaline Phosphatase (Alk Phos)**:
   - 44 to 147 IU/L

6. **SGOT (AST)**:
   - 10 to 40 units/L

7. **Triglycerides**:
   - Less than 150 mg/dL

8. **Platelets**:
   - 150,000 to 450,000 platelets mml/L

9. **Prothrombin Time (PT)**:
   - 11 to 13.5 seconds



## Possible interactions of the data points
___
1. **Bilirubin and Albumin**: These are both liver function tests. Bilirubin levels can increase due to liver dysfunction, while albumin levels can decrease. Their interaction might be indicative of the severity of liver disease.

2. **Ascites and Edema with Albumin**: Ascites and edema are clinical signs of advanced liver disease and can be influenced by the level of albumin, as it plays a crucial role in maintaining oncotic pressure in the blood vessels.

3. **Alkaline Phosphatase (Alk_Phos) and Bilirubin**: Both are markers of liver function. Elevated levels can indicate cholestasis or blockage of bile flow, often seen in liver diseases.

4. **SGOT and Bilirubin**: SGOT is an enzyme that can be elevated in liver damage. Together with bilirubin, these levels can indicate the extent of liver injury.

5. **Platelets and Prothrombin**: Both are involved in blood clotting. Liver disease can lead to thrombocytopenia (low platelet count) and prolonged prothrombin time, reflecting impaired liver synthesis function.

6. **Cholesterol and Triglycerides**: These are both lipids, and their interaction can be relevant in understanding the overall lipid profile, which is important in the context of cardiovascular risk and metabolic health.

7. **Drug (D-penicillamine) with Liver Function Tests**: The interaction between the use of D-penicillamine and liver function tests like Bilirubin, Albumin, Alk Phos might be insightful, especially in patients with Wilson’s Disease or liver involvement in Rheumatoid Arthritis.

8. **Age with Various Biomarkers**: Age might modulate the relationship between various biomarkers (like Bilirubin, Albumin, SGOT) and clinical outcomes.

##  Data table for reference
___
| Variable Name | Role    | Type        | Demographic | Description                                             | Units    | Missing Values |
|---------------|---------|-------------|-------------|---------------------------------------------------------|----------|----------------|
| ID            | ID      | Integer     |             | unique identifier                                       |          | no             |
| N_Days        | Other   | Integer     |             | number of days between registration and the earlier of  |          | no             |
|               |         |             |             | death, transplantation, or study analysis time in July 1986 |        |                |
| Status        | Target  | Categorical |             | status of the patient (C censored,                     |          | no             |
|               |         |             |             | CL (censored due to liver tx), or D (death))            |          |                |
| Drug          | Feature | Categorical |             | type of drug D-penicillamine or placebo                |          | yes            |
| Age           | Feature | Integer     | Age         | age                                                     | days     | no             |
| Sex           | Feature | Categorical | Sex         | M (male) or F (female)                                  |          | no             |
| Ascites       | Feature | Categorical |             | presence of ascites N (No) or Y (Yes)                   |          | yes            |
| Hepatomegaly  | Feature | Categorical |             | presence of hepatomegaly N (No) or Y (Yes)              |          | yes            |
| Spiders       | Feature | Categorical |             | presence of spiders N (No) or Y (Yes)                   |          | yes            |
| Edema         | Feature | Categorical |             | presence of edema N (no edema and no diuretic therapy   |          | no             |
|               |         |             |             | for edema), 0 (edema present without diuretics,         |          |                |
|               |         |             |             | or edema resolved by diuretics), or Y (edema despite    |          |                |
|               |         |             |             | diuretic therapy)                                       |          |                |
| Bilirubin     | Feature | Continuous  |             | serum bilirubin                                         | mg/dl    | no             |
| Cholesterol   | Feature | Integer     |             | serum cholesterol                                       | mg/dl    | yes            |
| Albumin       | Feature | Continuous  |             | albumin                                                 | gm/dl    | no             |
| Copper        | Feature | Integer     |             | urine copper                                            | ug/day   | yes            |
| Alk_Phos      | Feature | Continuous  |             | alkaline phosphatase                                    | U/liter  | yes            |
| SGOT          | Feature | Continuous  |             | SGOT                                                    | U/ml     | yes            |
| Triglycerides | Feature | Integer     |             | triglycerides                                           |          | yes            |
| Platelets     | Feature | Integer     |             | platelets per cubic                                     | ml/1000  | yes            |
| Prothrombin   | Feature | Continuous  |             | prothrombin time                                        | s        | yes            |
| Stage         | Feature | Categorical |             | histologic stage of disease (1, 2, 3, or 4)             |          | yes            |


In [ ]:
# column variables and normal ranges to print out  for EDA

normal_ranges = {
    "Bilirubin": (0.3, 1.2),
    "Cholesterol": (0, 200),
    "Albumin": (3.4, 5.4),
    "Copper": (70, 140),
    "Alk_Phos": (44, 147),
    "SGOT": (10, 40),
    "Tryglicerides": (0, 150),
    "Platelets": (150, 450),
    "Prothrombin": (9.5, 13.5)
}

columns_to_plot = [
    "Bilirubin",
    "Cholesterol",
    "Albumin",
    "Copper",
    "Alk_Phos",
    "SGOT",
    "Tryglicerides",
    "Platelets",
    "Prothrombin",
]

In [ ]:
def prelim_eda_histplot(df):
    """
    Create a histogram plot of each column in a pandas DataFrame using Plotly.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    
    columns_to_drop = ['id','Drug','Sex','Ascites','Hepatomegaly','Spiders','Edema','Stage','Status']
    columns_to_plot = [column for column in df.columns if column not in columns_to_drop]
    

    num_columns = len(columns_to_plot)
    num_rows = (num_columns - 1) // 4 + 1  # Calculate the number of rows based on the number of columns
    num_cols = min(num_columns, 4)  # Set the number of columns to 4 or the number of columns in the DataFrame, whichever is smaller

    # Create a subplot grid
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=columns_to_plot)

    # Loop over selected columns and create histogram plots in separate subplots
    for i, column in enumerate(columns_to_plot, start=1):
        row = (i - 1) // num_cols + 1
        col = (i - 1) % num_cols + 1
        trace = go.Histogram(x=df[column], nbinsx=20, name=column)
        fig.add_trace(trace, row=row, col=col)

    # Adjust layout
    fig.update_layout(height=300*num_rows, width=300*num_cols, title_text="Histograms of DataFrame Columns", showlegend=False, title_x=0.5)
    fig.update_traces(opacity=.75)
    fig.show()

# Example usage
prelim_eda_histplot(combined_train_df_imputed)

In [ ]:
def prelim_eda_histplot(df):
    """
    Create a histogram plot of each column in a pandas DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
    None
    """
    columns_to_plot = df.drop(['id','Drug','Sex','Ascites','Hepatomegaly','Spiders','Edema','Stage','Status'], axis=1)      


    num_columns = len(columns_to_plot.columns)
    num_rows = (num_columns - 1) // 4 + 1  # Calculate the number of rows based on the number of columns
    num_cols = min(num_columns, 4)  # Set the number of columns to 4 or the number of columns in the DataFrame, whichever is smaller
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 12))  # Adjust the figsize as needed

    # Loop over selected columns and create histogram plots in separate subplots
    for i, column in enumerate(columns_to_plot):
        row = i // num_cols
        col = i % num_cols
        sns.histplot(data=df, x=column, ax=axes[row, col], kde=True, bins=20)
        axes[row, col].set_title(f'{column}', fontsize=14)
        axes[row, col].set_aspect('auto')

    # Remove empty subplots (if any)
    for i in range(num_columns, num_rows * num_cols):
        fig.delaxes(axes.flatten()[i])

    # Adjust layout
    plt.tight_layout()
    plt.show()

prelim_eda_histplot(df_train)

In [ ]:

def create_boxplots(df, exclude_columns):
    """
    Create a series of boxplot subplots for the given DataFrame, excluding specified columns.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    exclude_columns (list): List of column names to exclude from plotting.

    Returns:
    matplotlib.figure.Figure: The figure object containing the subplots.
    """
    # Dropping the specified columns
    df_to_plot = df.drop(exclude_columns, axis=1)
    num_cols = len(df_to_plot.columns)
    
    # Determining the layout of the subplot grid
    cols_per_row = 3
    rows = (num_cols + cols_per_row - 1) // cols_per_row

    # Creating the subplots
    fig, axes = plt.subplots(rows, cols_per_row, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Flatten in case of a single row

    for i, col in enumerate(df_to_plot.columns):
        df_to_plot.boxplot(column=col, ax=axes[i])
        axes[i].set_title(col)

    # Hide unused subplots
    for j in range(i + 1, rows * cols_per_row):
        fig.delaxes(axes[j])

    plt.tight_layout()
    return fig

# Columns to exclude from the plot
exclude_columns = ['id', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage', 'Status']

# Creating the boxplot subplots
fig = create_boxplots(combined_train_df_imputed, exclude_columns)
plt.show()


In [ ]:

def create_boxplot_subplots(df, columns_to_plot, normal_ranges):
    """
    Create a subplot of box plots for the specified columns in the given DataFrame,
    including horizontal lines for normal ranges and corresponding legend entries.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    columns_to_plot (list): List of column names to plot.
    normal_ranges (dict): Dictionary of the form {column_name: (lower, upper)} representing normal ranges.

    Returns:
    None
    """
    fig = make_subplots(rows=3, cols=3, subplot_titles=columns_to_plot)

    # Add a box plot to each subplot cell and normal range lines
    for i, column in enumerate(columns_to_plot, start=1):
        row = (i - 1) // 3 + 1
        col = (i - 1) % 3 + 1
        fig.add_trace(go.Box(y=df[column], name=column), row=row, col=col)
        
        # Add normal range lines if available
        if column in normal_ranges:
            lower, upper = normal_ranges[column]
            # Lower normal range line
            fig.add_shape(type='line',
                          xref='x' + str(i), yref='y' + str(i),
                          x0=0, x1=1, y0=lower, y1=lower,
                          line=dict(color='green', width=2),
                          row=row, col=col)
            # Upper normal range line
            fig.add_shape(type='line',
                          xref='x' + str(i), yref='y' + str(i),
                          x0=0, x1=1, y0=upper, y1=upper,
                          line=dict(color='red', width=2),
                          row=row, col=col)
    
    # Add invisible traces for legend entries
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                             line=dict(color='green', width=2), name='Lower Normal Limit'))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                             line=dict(color='red', width=2), name='Upper Normal Limit'))

    fig.update_layout(
        height=1000,
        width=1000,
        title_text="Laboratory Measurements Distribution",
        font_size=14,
        title_x=0.5,
        legend=dict(bordercolor="Black", borderwidth=2)
    )

    fig.show()
create_boxplot_subplots(combined_train_df_imputed, columns_to_plot, normal_ranges)

In [ ]:

def create_plotly_express_heatmap(df, columns_to_plot, color_continuous_scale='viridis'):
    """
    Create a heatmap for the correlation matrix of specified columns in the given DataFrame using Plotly Express.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    columns_to_plot (list): List of column names to include in the heatmap.
    color_continuous_scale (str or list): Colormap used for the heatmap.

    Returns:
    plotly.graph_objs._figure.Figure: The figure object containing the heatmap.
    """
    # Select specified columns for correlation matrix
    df_subset = df[columns_to_plot]
    correlation_matrix = df_subset.corr()

    # Create the heatmap
    fig = px.imshow(
        correlation_matrix,
        labels=dict(x="Variable", y="Variable", color="Correlation"),
        x=columns_to_plot,
        y=columns_to_plot,
        color_continuous_scale=color_continuous_scale,
        zmin=-1,
        zmax=1,
        width=1000,
        height=1000,
        title="Heatmap of Correlation Matrix",
        template='plotly_white',
        color_continuous_midpoint=0
    )

    # Add annotations
    for i in range(len(columns_to_plot)):
        for j in range(len(columns_to_plot)):
            fig.add_annotation(
                x=columns_to_plot[i],
                y=columns_to_plot[j],
                text=f"{correlation_matrix.iloc[i, j]:.2f}",
                showarrow=False,
                font=dict(color="black", size=12),
            )

    # Update the layout
    fig.update_layout(
        title='Heatmap of Correlation Matrix',
        xaxis_nticks=len(columns_to_plot),
        yaxis_nticks=len(columns_to_plot),
        title_x=0.5
    )

    return fig
create_plotly_express_heatmap(df_train, columns_to_plot)


In [ ]:
def create_heatmap(df, figsize=(10, 8), annot=True, cmap='coolwarm',fmt='.1f'):
    """
    Create a heatmap for the correlation matrix of the numeric columns in the given DataFrame.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    figsize (tuple): Size of the heatmap.
    annot (bool): If True, write the data value in each cell.
    cmap (str): Colormap used for the heatmap.

    Returns:
    matplotlib.figure.Figure: The figure object containing the heatmap.
    """
    plt.figure(figsize=figsize)

    # Select only numeric columns for correlation matrix
    numeric_df = df.select_dtypes(include=[np.number])
    correlation_matrix = numeric_df.corr()

    sns.heatmap(correlation_matrix, annot=annot, cmap=cmap, fmt=fmt)
    plt.title('Heatmap of Correlation Matrix')
    plt.show()

# Example usage
create_heatmap(df_train)

## Outlier Detection
____
Some more indepth EDA on the outliers to get a better picture


In [ ]:
# Define the normal ranges for each test

# Create scatter plots
plt.figure(figsize=(25, 20))
for i, (test, range_vals) in enumerate(normal_ranges.items(), 1):
    plt.subplot(3, 3, i)
    custom_palette = ['#ff7f0e', '#2ca02c', '#1f77b4', '#d62728'] 
    sns.scatterplot(x=range(len(df_train)), y=df_train[test], hue=df_train['Stage'], alpha=0.3, palette=custom_palette)

    # Normal range lines
    plt.axhline(y=range_vals[0], color='black', linestyle='--', label='Lower Normal Limit')
    plt.axhline(y=range_vals[1], color='blue', linestyle='--', label='Upper Normal Limit')

    # Quantile range lines (25th and 75th percentiles)
    lower_quantile = df_train[test].quantile(0.25)
    upper_quantile = df_train[test].quantile(0.75)
    plt.axhline(y=lower_quantile, color='green', linestyle='-', label='25th Percentile')
    plt.axhline(y=upper_quantile, color='red', linestyle='-', label='75th Percentile')

    # Median line
    median = df_train[test].median()
    plt.axhline(y=median, color='black', linestyle='-', label='Median')


    plt.title(test)
    plt.xlabel("Index")
    plt.ylabel(test)
    plt.legend(bbox_to_anchor=(1.28,1),loc='upper right', fontsize='small', ncol=1, framealpha=0.1)

plt.tight_layout()
plt.show()

In [ ]:

fig = make_subplots(rows=3, cols=3, subplot_titles=list(normal_ranges.keys()))

color_map = {
    1: 'blue',
    2: 'green',
    3: 'red',
    4: 'purple'
}

# Loop over your normal_ranges to create scatter plots and lines
for i, (test, range_vals) in enumerate(normal_ranges.items(), start=1):
    row = (i - 1) // 3 + 1
    col = (i - 1) % 3 + 1

    # Add scatter plot
    for stage in df_train['Stage'].unique():
        stage_data = df_train[df_train['Stage'] == stage]
        fig.add_trace(
            go.Scatter(
                x=stage_data.index, 
                y=stage_data[test], 
                mode='markers', 
                name=f'Stage {stage}',
                marker=dict(color=color_map[stage], opacity=0.5,size=3),
                legendgroup=f'stage{stage}',  # Different group for each stage
                showlegend=(i == 1)  # Only show legend for the first subplot
            ),
            row=row, col=col
        )

    # Add normal range and quantile lines without adding to legend again
    fig.add_hline(y=range_vals[0], line_dash='solid', line_color='black', row=row, col=col)
    fig.add_hline(y=range_vals[1], line_dash='solid', line_color='lightblue', row=row, col=col)
    lower_quantile = df_train[test].quantile(0.25)
    upper_quantile = df_train[test].quantile(0.75)
    fig.add_hline(y=lower_quantile, line_dash='solid', line_color='orange', row=row, col=col)
    fig.add_hline(y=upper_quantile, line_dash='solid', line_color='red', row=row, col=col)

# Add invisible traces for the legend for lines
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines',
    line=dict(color='black', dash='solid'),
    legendgroup='lines',  # Same group for all lines
    showlegend=True, name='Lower Normal Limit'
))

fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines',
    line=dict(color='lightblue', dash='solid'),
    legendgroup='lines',
    showlegend=True, name='Upper Normal Limit'
))

fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines',
    line=dict(color='orange', dash='solid'),
    legendgroup='lines',
    showlegend=True, name='25th Percentile'
))

fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines',
    line=dict(color='red', dash='solid'),
    legendgroup='lines',
    showlegend=True, name='75th Percentile'
))

# Update layout to adjust the size and title
fig.update_layout(
    height=900, 
    width=1200, 
    title_text="Laboratory Measurements Distribution", 
    title_x=0.5,
    legend_traceorder='reversed'
)

# Show the figure
fig.show()

In [ ]:
# Assuming df_train is your DataFrame and normal_ranges is your dictionary of ranges

# Create a single scatter plot for demonstration
lower_normal_color = 'black'
upper_normal_color = 'black'
lower_normal_dash = 'dash'
quantile_25_color = 'blue'
quantile_75_color = 'green'
quantile_75_dash = 'solid'
line_width = 2

test = 'Bilirubin'
lower_limit = normal_ranges[test][0]
upper_limit = normal_ranges[test][1]

#needed to sort these so they showed up in the legned in the correct order
df_train_sorted = combined_train_df_imputed.sort_values(by='Stage', ascending=False)


fig = px.scatter(df_train_sorted, x=df_train_sorted.index, y=test, title=test, opacity=0.5,color='Stage',color_discrete_map={1:'green',2:'orange',3:'blue',4:'red'})

# Add horizontal lines for normal range and quantiles using variables
fig.add_hline(y=lower_limit, line_dash='solid', line_color=lower_normal_color, line_width=line_width)
fig.add_hline(y=upper_limit, line_dash=lower_normal_dash, line_color=upper_normal_color, line_width=line_width)
fig.add_hline(y=lower_quantile, line_dash='solid', line_color=quantile_25_color, line_width=line_width)
fig.add_hline(y=upper_quantile, line_dash=quantile_75_dash, line_color=quantile_75_color, line_width=line_width)
fig.add_hline(y=median, line_dash='solid', line_color='orange', line_width=line_width)

# Add invisible traces for the line legend using variables
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines', 
    line=dict(color=lower_normal_color, dash='dash', width=line_width), 
    name='Lower Normal Limit'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines', 
    line=dict(color=upper_normal_color, dash='solid', width=line_width), 
    name='Upper Normal Limit'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines',
    line=dict(color=quantile_25_color, dash='solid', width=line_width), 
    name='25th Percentile'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines', 
    line=dict(color=quantile_75_color, dash=quantile_75_dash, width=line_width), 
    name='75th Percentile'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='lines', 
    line=dict(color='orange', dash='solid', width=line_width), 
    name='Median'
))

# Update y-axis range to add a 10% margin on top
y_max = df_train_sorted[test].max()
y_margin = y_max * 0.1
fig.update_yaxes(range=[0, y_max + y_margin])
fig.update_traces(marker_coloraxis=None)   

# Show the figure
fig.show()

In [101]:
def convert_age_and_create_interactions(df):
    """
    Convert 'Age' from days to years and create interaction features.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.

    Returns:
    pandas.DataFrame: The modified DataFrame.
    """
    # Feature Engineering: Categorical Variable Interactions and Grouped Categorical Variables

    
    # df['Liver_Health_Indicators'] = df[['Ascites', 'Hepatomegaly', 'Spiders']].apply(lambda x: (x == 'Y').sum(), axis=1)

    df['Age_years'] = df['Age'] / 365.25
    df['Age_years'] = df['Age_years'].round().astype(int)
    
    # df['Age_group'] = pd.cut(df['Age_years'], bins=[25, 35, 45, 55, 65, 75, 80], labels=['1', '2', '3', '4', '5', '6'])
    # df['Age_group'] = pd.to_numeric(df['Age_group'])
    # df['Bilirubin_Albumin_interaction'] = df['Bilirubin'] * df['Albumin']
    # df['Platelets_Prothrombin_interaction'] = df['Platelets'] * df['Prothrombin']
    
    return df
pp_train = convert_age_and_create_interactions(combined_train_df_imputed)
test_df = convert_age_and_create_interactions(df_test)


In [102]:
pp_train.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,Age_years
0,4933.0,2255.0,D-penicillamine,22646.0,F,N,N,N,N,1.1,...,3.35,9.0,2520.0,92.00,209.0,309.0,11.0,1,C,62
1,3353.0,788.0,Placebo,12109.0,F,N,N,Y,N,6.4,...,3.79,186.0,2115.0,136.00,149.0,200.0,10.8,4,C,33
2,353.0,2698.0,Placebo,22646.0,F,N,Y,N,N,1.3,...,3.40,27.0,795.8,95.48,104.0,167.0,10.6,4,C,62
3,2667.0,1735.0,Placebo,12897.0,F,N,Y,N,N,0.6,...,3.64,48.0,794.0,52.70,214.0,271.0,10.6,3,C,35
4,3419.0,1783.0,Placebo,18713.0,F,N,N,Y,N,0.6,...,3.37,50.0,975.0,237.15,56.0,296.0,9.9,4,C,51


## Using Isolation Forest to detect outliers/anomalies

In [ ]:
from sklearn.ensemble import IsolationForest
anomaly_columns = [
    "Bilirubin",
    "Cholesterol",
    "Albumin",
    "Copper",
    "Alk_Phos",
    "SGOT",
    "Tryglicerides",
    "Platelets",
    "Prothrombin",
    "Age_years",
    # "Bilirubin_Albumin_interaction",
    # "Platelets_Prothrombin_interaction",
]

model_IF = IsolationForest(contamination=float(0.15),n_jobs=-1, random_state=42, verbose=1)
model_IF.fit(pp_train[anomaly_columns])
pp_train['anomaly_score_IF'] = model_IF.decision_function(pp_train[anomaly_columns])
pp_train['anomaly_IF'] = model_IF.predict(pp_train[anomaly_columns])

In [ ]:
def outlier_plot(data, outlier_method, x_var, y_var, x_axis_limits=[0,1], y_axis_limits=[0,1]):
    print(f'Outlier Method: {outlier_method}')
    print(f'Number of outliers: {len(data[data["anomaly_IF"] == -1])}')
    print(f'Number of inliers: {len(data[data["anomaly_IF"] == 1])}')
    print(f'Number of total points: {len(data)}')
    print(f'Percentage of outliers: {len(data[data["anomaly_IF"] == -1])/len(data)}')
    print('\n')


In [ ]:
palette = sns.color_palette(['#ff7f0e','#1f77b4'])

sns.pairplot(df_train, vars=anomaly_columns, hue='anomaly_IF', palette=palette)
plt.suptitle('Outlier Detection using Isolation Forest', y=1.1, fontweight='bold', fontsize=16)
plt.show()

In [ ]:
outlier_plot(pp_train, 'IF', 'Bilirubin', 'Cholesterol', [-1, 30],[-1, 220])

In [ ]:
#remove outliers
pp_train = pp_train.drop(pp_train[pp_train['anomaly_IF'] == -1].index)
# drop anomaly columns 
pp_train = pp_train.drop(['anomaly_score_IF','anomaly_IF'], axis=1)

# Feature Engineering: Converting Age from days to years and creating interaction features  and liver health indicators
___

In [103]:
pp_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8323 entries, 0 to 8322
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             8323 non-null   float64
 1   N_Days         8323 non-null   float64
 2   Drug           8323 non-null   object 
 3   Age            8323 non-null   float64
 4   Sex            8323 non-null   object 
 5   Ascites        8323 non-null   object 
 6   Hepatomegaly   8323 non-null   object 
 7   Spiders        8323 non-null   object 
 8   Edema          8323 non-null   object 
 9   Bilirubin      8323 non-null   float64
 10  Cholesterol    8323 non-null   float64
 11  Albumin        8323 non-null   float64
 12  Copper         8323 non-null   float64
 13  Alk_Phos       8323 non-null   float64
 14  SGOT           8323 non-null   float64
 15  Tryglicerides  8323 non-null   float64
 16  Platelets      8323 non-null   float64
 17  Prothrombin    8323 non-null   float64
 18  Stage   

In [ ]:
def master_preprocessing(df):
    """
    Apply all preprocessing steps to the dataframe.

    :param df: DataFrame to be processed
    :param features_to_transform: List of features to transform outliers in
    :return: Preprocessed DataFrame
    """
    # Apply preprocessing for new features
    df = convert_age_and_create_interactions(df)
    return df

In [104]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.compose import ColumnTransformer
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll import scope

# Splitting the dataset into training and testing sets
pp_train, pp_test = train_test_split(pp_train, test_size=0.2, random_state=42)

# Encoding the target variable 'Status' if it's categorical
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(pp_train['Status'])
y_test = label_encoder.transform(pp_test['Status'])

# Selecting categorical columns (excluding the target variable 'Status')
cat_cols = pp_train.select_dtypes(include=['object']).drop(columns=['Status']).columns

# Preprocessing with OneHotEncoder and StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), pp_train.select_dtypes(exclude=['object']).drop(['id'], axis=1).columns),
        ('cat', OneHotEncoder(), cat_cols)
    ])

# Fit and transform the training data and transform the test data
X_train = preprocessor.fit_transform(pp_train.drop(columns=['Status', 'id']))
X_test = preprocessor.transform(pp_test.drop(columns=['Status', 'id']))

# Define the space of hyperparameters to search
space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 8, 1)),
    'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 1, 10),
    'gamma': hp.loguniform('gamma', -10, 10),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'random_state': 42,
}
def objective(params):

    xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=3, eval_metric='mlogloss', **params)
    xgb_model.fit(X_train, y_train)
    y_pred_proba = xgb_model.predict_proba(X_test)
    logloss = log_loss(y_test, y_pred_proba)  # Assuming you've imported log_loss from sklearn.metrics
    return {'loss': logloss, 'status': STATUS_OK}

# Run the hyperparameter search using the tpe algorithm
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=4000,  # Adjust based on your computation resource
            trials=Trials())

print("Best parameters: ", best)



100%|██████████| 100/100 [00:49<00:00,  2.00trial/s, best loss: 0.44424217051405196]
Best parameters:  {'colsample_bytree': 0.5031331406523033, 'gamma': 0.00012620421737027032, 'learning_rate': 0.1150005120091401, 'max_depth': 6.0, 'min_child_weight': 0.335896630953778, 'reg_alpha': 3.4441782632180264, 'reg_lambda': 6.104907325210941, 'subsample': 0.9846733008892326}


In [105]:
best['max_depth'] = int(best['max_depth'])
final_model = xgb.XGBClassifier(n_estimators = 750, objective='multi:softprob', num_class=3, eval_metric='mlogloss', **best, early_stopping_rounds=50)
final_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

# Predict class labels
y_pred_class = final_model.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred_class)
print("Final model accuracy: ", final_accuracy)

# Predict probabilities and calculate log loss
y_pred_proba = final_model.predict_proba(X_test)
final_logloss = log_loss(y_test, y_pred_proba)
print("Final model log loss: ", final_logloss)

[0]	validation_0-mlogloss:1.01886
[100]	validation_0-mlogloss:0.44303
[162]	validation_0-mlogloss:0.44363
Final model accuracy:  0.8312312312312312
Final model log loss:  0.4418360456626488


In [ ]:
# Transform the test data using the same preprocessor
X_new = preprocessor.transform(test_df.drop(columns=['id']))

# Making predictions on the new data
predictions = final_model.predict_proba(X_new)

# Creating the DataFrame for submission
prediction_columns = ['Status_C', 'Status_CL', 'Status_D']
submission_df = pd.DataFrame(predictions, columns=prediction_columns)
submission_df['id'] = test_df['id']

# Reordering columns to make 'id' the first column
submission_df = submission_df[['id'] + prediction_columns]

# Saving the DataFrame to a CSV file
submission_df.to_csv('../data/submission.csv', index=False)

In [ ]:
submission_df.head()